In [1]:
import requests
import pandas as pd
import ast
import re
import json

In [2]:
XAI_API_URL = "https://api.x.ai/v1/chat/completions"

In [3]:
def get_api_key(file_path):
    with open(file_path, "r") as file:
        return file.read().strip()

In [4]:
def extract_valid_list(response_text):
    
    response_text = re.sub(r"```[a-zA-Z]*\n", "", response_text)  
    response_text = response_text.strip("`")  

    match = re.search(r"\[[^\]]+\]", response_text, re.DOTALL)  

    if match:
        list_text = match.group(0)
        try:
            extracted_list = ast.literal_eval(list_text)  
            if isinstance(extracted_list, list):
                return extracted_list
        except Exception as e:
            print(f"Error parsing response: {e}")  
            return []  
    
    return []

In [5]:
def generate_ner_tags(sentence, api_key):

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "grok-2-latest",
        "messages": [
            {"role": "system", "content": "You are an AI that outputs only valid Python lists. No explanations."},
            {"role": "user", "content": f"""
            You are a highly intelligent AI trained for named entity recognition.
            Your task is to assign the most suitable ner_tags to every token in the following text based on the provided list of possible ner_tags.
            
            Assign NER tags from the set:
            ['B-Further Specification', 'I-Further Specification', 'O', 'I-XOR Gateway',
            'B-XOR Gateway', 'B-Activity Data', 'B-Condition Specification', 'I-Actor',
            'B-AND Gateway', 'I-Activity','B-Activity', 'I-Activity Data',
            'I-Condition Specification', 'I-AND Gateway', 'B-Actor']
            to each token in:
            {sentence}

            Output ONLY a Python-style list of NER tags, e.g.:
            ['O', 'O', 'B-Actor', 'I-Actor']

            No explanations. No additional text. Do not include ```python or any Markdown formatting.
            """}
        ],
        "temperature": 0.0,
        "max_tokens": 1024
    }

    try:
        response = requests.post(XAI_API_URL, headers=headers, data=json.dumps(payload))
        response_data = response.json()

        if "choices" in response_data and response_data["choices"]:
            response_text = response_data["choices"][0]["message"]["content"].strip()
            print(f"DEBUG: Raw API Response: {response_text}")  # Debugging
            return extract_valid_list(response_text)
        else:
            print("DEBUG: Empty response from Grok API")
            return []
    
    except Exception as e:
        print(f"Error calling Grok API: {e}")
        return []

In [6]:
def process_dataset(input_file, output_file, api_key_file):

    api_key = get_api_key(api_key_file)
    dataset = pd.read_csv(input_file)

    def process_row(row):
        try:
            tokens = ast.literal_eval(row["tokens"])  
            sentence = " ".join(tokens)
            return generate_ner_tags(sentence, api_key)
        except Exception as e:
            return f"Error: {str(e)}"

    dataset["grok-2-latest_ner_tags"] = dataset.apply(process_row, axis=1)

    dataset.to_csv(output_file, index=False)
    print(f"Updated dataset saved to {output_file}")

In [7]:
input_file = "PET-Customized.csv" 
output_file = "PET-Customized-Grok2Vision.csv" 
api_key_file = "./API_Key/xAI.txt"

if __name__ == "__main__":
    process_dataset(input_file, output_file, api_key_file)

DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O', 'O']
DEBUG: Empty response from Grok API
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O', 'B-Actor', 'O', 'O', 'B-Actor', 'O', 'O']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O', 'O']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'B-Actor', 'I-Actor', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Activity', 'I-Activity', 'O', 'O', 'O